# Figures for YanLing

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 250

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')

## Figure of main populations

In [ ]:
# Join al subpopulations into one
major_merged_types = ['Fibroblast', 'Keratinocyte', 'Neutrophil', 'Perivascular cell', 'T cell']

In [ ]:
merged_cell_types = adata_all_harmony.obs['cell_type_whole'].astype(str)

for mct in major_merged_types:
    mask = [mct in i for i in adata_all_harmony.obs['cell_type_whole']]
    merged_cell_types[mask] = mct

merged_cell_types[['Acan' in i for i in adata_all_harmony.obs['cell_type_whole']]] = 'Fibroblast $Acan^+$' 

merged_cell_types[[('B cell' in i) or ('T cell' in i) or ('NK cell' in i) or ('Plasma cell' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'Lymphoid' 
merged_cell_types[[(('Fibroblast' in i) and ('Acan' not in i)) or ('Skeletal' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'Fibroblast' 
merged_cell_types[[('Monocyte' in i) or ('Dendritic' in i) or ('Macrophage' in i) for i in adata_all_harmony.obs['cell_type_whole']]] = 'APC' 



adata_all_harmony.obs['merged_cell_type'] = merged_cell_types

dict_cell_types_colors = {'Fibroblast $Acan^+$': '#d38d5f', 
                          'Neutrophil': '#5599ff',
                          
                          'Fibroblast': '#a05a2c',
                          'APC': '#87decd',
                          
                          'Perivascular cell': '#ff9955',
                          'Langerhans cell': '#87d283',
                          
                          'Keratinocyte': '#338000',
                          'Lymphoid': '#5f5fd3',
                          
                          'Endothelial': '#d40055',
                          'Schwann cell': '#d3bc5f',
                          
                          'Lymphatic': '#ff5599',
                          'Mast cell': '#442178',
                          
                          'Red blood cell': '#800033',
                    }


adata_all_harmony.obs['merged_cell_type'] = adata_all_harmony.obs['merged_cell_type'].astype('category')
adata_all_harmony.uns['merged_cell_type_colors'] = [dict_cell_types_colors[i] for i in adata_all_harmony.obs['merged_cell_type'].cat.categories]

In [ ]:
sc.pl.umap(adata_all_harmony, color='merged_cell_type')

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
s = 8
plt.tight_layout()
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'KOD11'], color='merged_cell_type', s=s, ax = axs[0, 0], legend_loc=False, title='KOD11', frameon=False, show=False)
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'KOD12'], color='merged_cell_type', s=s, ax = axs[0, 1], legend_loc=False, title='KOD12', frameon=False, show=False)

sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'WT1'], color='merged_cell_type', s=s, ax = axs[1, 0], legend_loc=False, title='WT1', frameon=False, show=False)
sc.pl.umap(adata_all_harmony[adata_all_harmony.obs['batch'] == 'WT2'], color='merged_cell_type', s=s, ax = axs[1, 1], legend_loc=False, title='WT2', frameon=False,  show=False)

plt.subplots_adjust(wspace=0.025, hspace=0.1)

fig2, ax = plt.subplots(1, 1)
plt.axis('off')
legend_elements = [mpl.lines.Line2D([0], [0], marker='o', color=(1, 1, 1, 1), label=name_i, markerfacecolor=color_i, markersize=15) for name_i, color_i in dict_cell_types_colors.items()]
ax.legend(handles=legend_elements, loc='upper left', ncol=7, frameon=False, fontsize=25)

plt.show()